In [ ]:
import pandas as pd
df_train= pd.read_csv("Data/train_data.csv")

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')

In [ ]:
#Importing necessary libraries
import nltk
import pandas as pd
from textblob import Word
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer, one_hot
from keras.utils import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split

#Pre-Processing the text
def cleaning(df, stop_words):
    df_train['Review'] = df_train['FFT answer'].apply(lambda x: ' '.join(x.lower() for x in str(x).split()))
    # Replacing the digits/numbers
    df['Review'] = df['Review'].str.replace('d', '')
    # Removing stop words
    df['Review'] = df['Review'].apply(lambda x: ' '.join(x for x in x.split() if x not in stop_words))
    # Lemmatization
    df['Review'] = df['Review'].apply(lambda x: ' '.join([Word(x).lemmatize() for x in x.split()]))
    return df

stop_words = stopwords.words('english')
data_cleaned = cleaning(df_train, stop_words)

#Generating Embeddings using tokenizer
tokenizer = Tokenizer(num_words=500, split=' ')
tokenizer.fit_on_texts(data_cleaned['Review'].values)
X = tokenizer.texts_to_sequences(data_cleaned['Review'].values)
X = pad_sequences(X)

# one hot encoding on target
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
Y= df_train["Comment sentiment"].values
Y= enc.fit_transform(Y.reshape(-1,1)).toarray()

# train test split
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=5)


In [ ]:
#Model Building
from keras import optimizers
model = Sequential()
model.add(Embedding(500, 160, input_length = X.shape[1]))
model.add(SpatialDropout1D(0.3))
model.add(LSTM(512, dropout=0.3, recurrent_dropout=0.3))
model.add(Dense(256, activation='LeakyReLU'))
model.add(Dense(128, activation='LeakyReLU'))
model.add(Dense(6, activation='softmax'))

adam = optimizers.Adam(lr=0.01, weight_decay=1e-6, ema_momentum=0.9, use_ema=True)
model.compile(loss = 'categorical_crossentropy', optimizer=adam, metrics = ['accuracy'])
print(model.summary())

#Model Training
model.fit(X_train, Y_train, epochs = 20, batch_size=32, verbose =1)
#Model Testing
score, acc = model.evaluate(X_test,Y_test)
print(score)
#print(acc)

#result
#https://the-strategy-unit.github.io/PatientExperience-QDC/background/text_mining_final_report_mar-22.pdf

In [ ]:
#model.save('model.h5')